In [ ]:
%run ../prelude.py

## Create an adjancency matrix

In [ ]:
adjmat = Tensor.fromRandom(["S", "D"], [10, 10], (1.0, 0.3), 1, seed=10, name="AdjMat")

displayTensor(adjmat)

## Create a frontier

In [ ]:
frontier = Tensor.fromRandom(["S"], [10], (0.4,), 1, seed=20, name="Frontier1")

displayTensor(frontier)

## Create reference output using untiled traversal

In [ ]:
# Note that S and D are really both V, so we can swap the names for clarity
next_frontier_verify = Tensor(rank_ids=["S"], name="NextFrontier_Verify")

frontier_s = frontier.getRoot()
adjmat_s = adjmat.getRoot()
next_frontier_verify_s = next_frontier_verify.getRoot()

for s, (_, adjmat_d) in frontier_s & adjmat_s:
    for d, (next_frontier_verify_ref, _) in next_frontier_verify_s << adjmat_d:
        next_frontier_verify_ref <<= 1

displayTensor(next_frontier_verify)
        

## Pre-tile adjacency matrix (concordant for source-stationary)

In [ ]:
S0 = 5
D0 = 5

adjmat_tiled = adjmat.splitUniform(D0, depth=1).splitUniform(S0).swapRanks(1)
adjmat_tiled.setName("AdjMatTiled")

displayTensor(adjmat_tiled)

### Swapped representation (concordant for destination-stationary)

In [ ]:
adjmat_swapped = adjmat_tiled.swapRanks(2).swapRanks(0)
adjmat_swapped.setName("AdjMatTiledSwapped")
displayTensor(adjmat_swapped)

## Pre-process the frontier into tiles

In [ ]:
frontier_tiled = frontier.splitUniform(S0)
frontier_tiled.setName("FrontierTiled")

displayTensor(frontier_tiled)

## Source-Stationary Tiled Advance Kernel

In [ ]:
# Output frontier is produced tiled (concordant)
next_frontier_tiled = Tensor(rank_ids=["S1", "S0"], name = "NextFrontierTiled")

adjmat_s1 = adjmat_tiled.getRoot()
frontier_s1 =  frontier_tiled.getRoot()
next_frontier_s1 = next_frontier_tiled.getRoot()

canvas = createCanvas(frontier_tiled, adjmat_tiled, next_frontier_tiled)

for s1, (frontier_s0, adjmat_d1) in frontier_s1 & adjmat_s1:
    for d1, (next_frontier_s0, adjmat_s0) in next_frontier_s1 << adjmat_d1:
        for s0, (_, adjmat_d0) in frontier_s0 & adjmat_s0:
            for d0, (next_frontier_ref, _) in next_frontier_s0 << adjmat_d0:
                next_frontier_ref <<= 1
                addFrame(canvas, (s1, s0), (s1, d1, s0, d0), (d1, d0))
        
displayCanvas(canvas)
assert next_frontier_tiled.flattenRanks(coord_style="absolute").getRoot() == next_frontier_verify.getRoot()

## Destination-Stationary Tiled Advance Kernel

In [ ]:
# Output frontier is produced tiled (concordant)
next_frontier_tiled = Tensor(rank_ids=["S1", "S0"], name = "NextFrontierTiled")

adjmat_d1 = adjmat_swapped.getRoot()
frontier_s1 =  frontier_tiled.getRoot()
next_frontier_s1 = next_frontier_tiled.getRoot()

canvas = createCanvas(frontier_tiled, adjmat_swapped, next_frontier_tiled)

for d1, (next_frontier_s0, adjmat_s1) in next_frontier_s1 << adjmat_d1:
    for s1, (frontier_s0, adjmat_d0) in frontier_s1 & adjmat_s1:
        for d0, (next_frontier_ref, adjmat_s0) in next_frontier_s0 << adjmat_d0:
            for s0, (_, _) in frontier_s0 & adjmat_s0:
                next_frontier_ref <<= 1
                addFrame(canvas, (s1, s0), (d1, s1, d0, s0), (d1, d0))
                break # This break is interesting - it represents the "OR" of the intersection.
                      # Note that if this loop was in parallel we wouldn't care which particular
                      # coordinate passed intersection, only that one of them did.
        
displayCanvas(canvas)
assert next_frontier_tiled.flattenRanks(coord_style="absolute").getRoot() == next_frontier_verify.getRoot()